In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:16pt;}
div.text_cell_render.rendered_html{font-size:16pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:16pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
table.dataframe{font-size:16px;}
</style>
"""))

# <b><font size="7" color="red">ch14. 웹데이터수집</font></b>
# 1장 BeautifulSoup과 parser
'pip install bs4 아나콘다를 설치하면 자동
- https://beautiful-soup-4.readthedocs.io/en/latest/

In [6]:
# 내 local PC의 파일을 url처럼 접근
import requests # http 요청 처리하는 Lib
from requests_file import FileAdapter # file://프로토콜을 다루기 위함

In [3]:
s = requests.Session() # HTTP 요청처리하는 세션 객체
s.mount("file://", FileAdapter())
# file:// == c:(로컬파일)을 접근
response = s.get('file:///taro/ai/source/01_python/data/ch14_sample.html')
response

<Response [200]>

In [4]:
if response:
    print('잘 접근하였습니다.')
else:
    print('접근을 못했습니다')    

잘 접근하였습니다.


In [5]:
response.status_code

200

In [6]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [7]:
response.text # HTML파일의 텍스트 내용

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [7]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, # response.text
                    "html.parser")
# soup

AttributeError: 'HTTPResponse' object has no attribute 'content'

In [16]:
# soup.select_one('선택자') : 해당선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print(el)
print(el.text)
print(el.string)
print('el의 속성들 :', el.attrs)
print('el의 id속성 :', el.attrs['id']) # el.attrs은 딕셔너리
print('el의 id속성 :', el.attrs.get('id'))
print('el의 href속성 :', el.attrs.get('href'))
print('el의 태그이름 :', el.name)

<h1 class="greeting css" id="text">Hello, CSS</h1>
Hello, CSS
Hello, CSS
el의 속성들 : {'class': ['greeting', 'css'], 'id': 'text'}
el의 id속성 : text
el의 id속성 : text
el의 href속성 : None
el의 태그이름 : h1


In [19]:
# soup.select('선택자') : 해당 선택자 모두
els = soup.select('h1')
print(els)
print('els의 text들 :', [el.text for el in els])
print('els의 string들 :', [el.string for el in els])
print('els의 속성들 :', [el.attrs for el in els])
print('els의 class 속성들 :', [el.attrs.get('class') for el in els])

[<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
els의 text들 : ['Hello, CSS', 'Hi, CSS']
els의 string들 : ['Hello, CSS', 'Hi, CSS']
els의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text'}, {'class': ['css']}]
els의 class 속성들 : [['greeting', 'css'], ['css']]


In [20]:
# soup.find(태그, 속성) : soup.select_one('선택자')와 유사
print('select_one :', soup.select_one('h1.css') )
print('find :', soup.find('h1', {'class':'css'}) ) # soup.select_one('h1.css')
print('find :', soup.find('h1', class_='css') )
print()
print('select_one :', soup.select_one('h1#text') )
print('find :', soup.find('h1', {'id':'text'}))

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>


In [21]:
# soup.find_all(태그, 속성) : soup.select('선택자')와 유사
print('select :', soup.select('h1.css'))
print('find_all :', soup.find_all('h1', class_='css'))
print('find_all :', soup.find_all('h1', {'class':'css'}))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [22]:
print('select :', soup.select('h1.css, span'))
print('find_all :', soup.find_all(['h1', 'span'], {'class':'css'}))
print('find_all :', soup.find_all('h1', class_='css') + \
                     soup.find_all('span'))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]


In [23]:
# 없는 엘리먼트 찾기
print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))
print('find_all(빈 list) :', soup.find_all('a', {'class', 'css'}))
print('find(None) :', soup.find('a', {'class':'css'}))

select(빈 list) : []
select_one(None) : None
find_all(빈 list) : []
find(None) : None


# 2장. 정적 웹 데이터 수집(정적 웹크롤링)
## 2.1 BeautifulSoup을 활용한 html 웹 데이터수집
### 1) 환율정보 가져오기 ( 네이버증권 -> 시장지표)
https://finance.naver.com/marketindex/

In [ ]:
# 크롤링 허용범위는 사이트마다 ~/robots.txt에서 확인
    # Allow : / - 사이트의 모든 경로(/)에 대한 크롤링 허용

In [26]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
# response.status_code
# response.text / response.content.decode('cp949')
soup = BeautifulSoup(response.text, "html.parser")

In [30]:
# 방법2
from urllib.request import urlopen
url = 'https://finance.naver.com/marketindex/'
response = urlopen(url)
response.status
# response.read().decode('cp949')
soup = BeautifulSoup(response, 'html.parser')

In [34]:
# div.head_info > span.value(find함수)
prices=[]
headinfos = soup.find_all('div', class_='head_info')
for headinfo in headinfos:
    price = headinfo.find('span', class_='value')
#    print(price.text.replace(',', ''))
    prices.append(float(''.join(price.text.split(','))))
print(prices)

[1454.5, 945.19, 1681.33, 204.2, 153.42, 1.1564, 1.3161, 99.47, 59.75, 1698.27, 4009.8, 189125.23]


In [45]:
# div.head_info > span,value (select함수)
prices = soup.select('div.head_info > span.value')
# [float(p.text.replace(',','')) for p in prices]

In [36]:
titles = soup.select('h3.h_lst > span.blind')
for t in titles:
    print(t.text, end='\t')

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [37]:
units = soup.select('div.head_info > span > span.blind')
units = [unit.text for unit in units]
units.insert(7, '') # 7번째에 ''추가
units

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [41]:
statuses = soup.select('div.head_info > span.blind')
for idx in range(len(status)):
    print(status[idx].text, end='\t')

하락	하락	하락	하락	상승	하락	상승	하락	상승	상승	상승	상승	

In [46]:
len(titles), len(prices), len(units), len(statuses)

(12, 12, 12, 12)

In [ ]:
for idx in range(len(titles)):
    print(''{} : {}{})

In [47]:
for title, price, unit, status in zip(titles, prices, units, statuses):
    print('{}:{}{} -{}'.format(title.text, price.text, unit, status.text))

미국 USD:1,454.50원 -하락
일본 JPY(100엔):945.19원 -하락
유럽연합 EUR:1,681.33원 -하락
중국 CNY:204.20원 -하락
달러/일본 엔:153.4200엔 -상승
유로/달러:1.1564달러 -하락
영국 파운드/달러:1.3161달러 -상승
달러인덱스:99.4700 -하락
WTI:59.75달러 -상승
휘발유:1698.27원 -상승
국제 금:4009.8달러 -상승
국내 금:189125.23원 -상승


### 2) 이번주 로또 번호 출력
- https://www.dhlottery.co.kr/gameResult.do?method=byWin 로또번호 당첨번호 사이트
'''
1197회(2025년 11월 08일 추첨)
당첨번호 [1, 5, 7, 26, 28,

In [49]:
from urllib.request import urlopen
url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
response = urlopen(url)
soup = BeautifulSoup(response, 'html.parser')
response.status

200

In [67]:
times = soup.select_one('div.win_result strong').text # 1197회
date  = soup.select_one('div.win_result > p.desc').text
# date = soup.find('p', class_='desc').text
lotto_number_el = soup.select('div.num.win span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(soup.select_one('div.num.bonus > p > span').text)

In [68]:
print(times, date)
print('당첨번호 ',lotto_number)
33print('보 너 스 ', bonus_number)

1197회 (2025년 11월 08일 추첨)
당첨번호  [1, 5, 7, 26, 28, 43]
보 너 스  30


### 3) 다음 검색 리스트
```
no title          href
0 "한풀 꺾인 비트코인
```

In [78]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx,
                          'title':item.text,
                          'link':item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)    

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인


,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
3,3,“지금 들어가?”… 트럼프 관세 배당 발언에 10만달러 회복한 비트코인,http://v.daum.net/v/20251110114112103
4,4,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
5,5,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
6,6,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
7,7,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
8,8,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
9,9,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909


In [79]:
# 방법2 : 불가
from urllib.request import urlopen
from urllib.parse import quote
word = quote('비트코인') # url에 한글이 포함된 경우 한글을 quote()로 변환
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
response = urlopen(url)
print(url)
soup = BeautifulSoup(response, "html.parser")

items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx,
                          'title':item.text,
                          'link':item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list) 

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


""


In [80]:
items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs['href']])
import pandas as pd
pd.DataFrame(items_find_list, columns=['번호', '제목', '링크'])   

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인


,번호,제목,링크
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [81]:
items_find_list = [] # 검색한 결과를 담을 리스트
# div.item-title > strong.tit-g > a
item_titles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a = item.find('a')
    #print(idx, a.text, a.attrs['href'])
    #items_find_list.append([idx, a.text, a.attrs['href']])
    items_find_list.append({
        'no':idx,
        'title':a.text,
        'link':a.attrs['href'],
    })
pd.DataFrame(items_find_list)

,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [90]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력->list를 return'
    #url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    # response = requests.get(url)
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, "html.parser")
    
    items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
    item_titles = soup.select('div.item-title > strong.tit-g > a')
    for idx, item in enumerate(item_titles):
        items_find_list.append([(page-1)*10+idx, item.text, item.attrs['href']])
    return items_find_list

In [91]:
collect_news_list('청바지', 2)

[[10,
  ' 167cm 김설현, 흰 티에 청바지 또 레전드 썼네…원조 광고퀸 포스 눈길 ',
  'http://v.daum.net/v/20251031132450232'],
 [11,
  ' "상견례서 청바지 입고 온 예비 시부모, 우리 집 무시하나…심란하다" ',
  'http://v.daum.net/v/20251030050113673'],
 [12,
  ' 송가인, 날씬한 청바지핏으로 시선 집중! “크리스마스 트리와 함께” ',
  'http://v.daum.net/v/20251108213228200'],
 [13,
  ' 상견례 때 예비 시부모 청바지 입고 나오면 상대 무시? 갑론을박 ',
  'http://v.daum.net/v/20251030141223976'],
 [14,
  " 패피들이 가을에 청바지보다 자주 입는다는 '이 컬러' 팬츠 ",
  'http://v.daum.net/v/20251031180546469'],
 [15, ' 청바지와 이 자켓이 가을 룩 필승 조합이래요 ', 'http://v.daum.net/v/20251025101225501'],
 [16,
  " 땀 뻘뻘 운동하면서도 입는다...케네디, 지독한 '청바지 집착' 왜 ",
  'http://v.daum.net/v/20251018070139949'],
 [17,
  ' 여름 내내 입던 청바지 가을에 티 안나게 재탕하는 법 ',
  'http://v.daum.net/v/20250924114747275'],
 [18, ' 아이브 안유진, 완벽한 청바지 핏 ', 'http://v.daum.net/v/20251028173148422'],
 [19,
  ' 강민경, 역시 사복 여신…진짜 마르면 나오는 청바지 핏 ',
  'http://v.daum.net/v/20251030000223734']]

In [95]:
import time
keywords = ['청바지', '동대문']
result0 = [] # 청바지 1~3페이지까지 검색한 결과를 담을 list
result1 = [] # 동대문 1~3페이지까지 검색한 결과를 담을 list
pages = 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번째 검색어 {keyword} 검색 결과 수집 중입니다===')
    for page in range(1, pages+1):
        if i==0:
            result0.extend(collect_news_list(keyword, page))
        elif i==1:
            result1.extend(collect_news_list(keyword, page))
        time.sleep(0.5)

===1번째 검색어 청바지 검색 결과 수집 중입니다===
===2번째 검색어 동대문 검색 결과 수집 중입니다===


In [96]:
result0_df = pd.DataFrame(result0, columns=['no', 'title','link'])
result1_df = pd.DataFrame(result1, columns=['no', 'title','link'])
result1_df.head()

,no,title,link
0,0,"“돈 벌어가세요” 동대문 일요시장, 겨울옷 핫플된 사연? [르포]",http://v.daum.net/v/20251110100357735
1,1,"동대문구, ‘11월 자전거교실’ 개강… 올해 마지막 초보자 교육 진행",http://v.daum.net/v/20251110145114101
2,2,"음주운전 차, 동대문역서 인도 돌진...日 관광객 '사망'",http://v.daum.net/v/20251103073316480
3,3,"동대문구, 25일까지 '주민 건강도시학교'···""건강하게 살자""",http://v.daum.net/v/20251109223318356
4,4,"서울 동대문구, 15일 배봉산정원 가족축제 개최",http://v.daum.net/v/20251104104517122


In [99]:
result0_df.to_csv(f'data/ch14_{keywords[0]}.csv', index=False, encoding='cp949')
result1_df.to_csv(f'data/ch14_{keywords[1]}.csv', index=False)

### 4) User-Agent를 추가하여 크롤링
- 자동으로 브라우저를 통해 요청하는 것처럼 보이기

In [5]:
# 방법2
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인')
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
# headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
# request = Request(url, headers=headers)
request = Request(url)
request.add_header('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36')
response = urlopen(request)
soup = BeautifulSoup(response, 'html.parser')
soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


NameError: name 'BeautifulSoup' is not defined

- https://www.melon.com/chart/index.htm
    - https://www.melon.com/robots.txt 에서 User-Agent에 봇이 지정

In [42]:
# 방법1
import requests
url = 'https://www.melon.com/chart/index.htm'
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, 'html.parser')
# soup
ranks = soup.select('td div.wrap > span.rank')
titles = soup.select('div.ellipsis.rank01 > span')# title.text.strip()
singers = soup.select('div.ellipsis.rank02')# singers.text.strip()[:20] 
for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1위 | Blue Valentine - NMIXXNMIXX
2위 | 타임캡슐 - 다비치다비치
3위 | Golden - HUNTR/X, EJAE, AUDRE
4위 | Good Goodbye - 화사 (HWASA)화사 (HWASA)
5위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
6위 | 달리 표현할 수 없어요 - 로이킴로이킴
7위 | Drowning - WOODZWOODZ
8위 | 뛰어(JUMP) - BLACKPINKBLACKPINK
9위 | 어제보다 슬픈 오늘 - 우디 (Woody)우디 (Woody)
10위 | Soda Pop - KPop Demon Hunters C
11위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)박다혜
12위 | FAMOUS - ALLDAY PROJECTALLDAY
13위 | 모르시나요(PROD.로코베리) - 조째즈조째즈
14위 | XOXZ - IVE (아이브)IVE (아이브)
15위 | IRIS OUT - Kenshi YonezuKenshi 
16위 | 한번 더 이별 - 이창섭이창섭
17위 | Rich Man - aespaaespa
18위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴로이킴
19위 | 너에게 닿기를 - 10CM10CM
20위 | Hollywood Action - BOYNEXTDOORBOYNEXTDO
21위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGONG-DRAGON
22위 | 운명 (2025) - 먼데이 키즈, 이이경먼데이 키즈, 이
23위 | body - 다영 (DAYOUNG)다영 (DAYO
24위 | Whiplash - aespaaespa
25위 | toxic till the end - 로제 (ROSÉ)로제 (ROSÉ)
26위 | 천상연 - 이창섭이창섭
27위 | 시작의 아이 - 마크툽 (MAKTUB)마크툽 (MAK
28위 | 청춘만화 - 이무진이무진
29위 | 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 - AKMU

In [8]:
# 방법2 : User-Agent 추가
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/index.htm'
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
melonpage = urlopen(request)
soup = BeautifulSoup(melonpage, 'html.parser')
# soup


In [9]:
# 방법1 : User-Agent 추가
url = 'https://www.melon.com/chart/index.htm'
headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melonpage.content
                    'html.parser')

ranks = soup.select('td div.wrap.t_center > span.rank')
titles = soup.select('div.ellipsis.rank01 > span') # title.text.strip()
singers = soup.select('div.ellipsis.rank02') # singer.text.strip()[:20]
# 1위 | 노래제목 - 가수명
for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1위 | Blue Valentine - NMIXXNMIXX
2위 | 타임캡슐 - 다비치다비치
3위 | Golden - HUNTR/X, EJAE, AUDRE
4위 | 달리 표현할 수 없어요 - 로이킴로이킴
5위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
6위 | Good Goodbye - 화사 (HWASA)화사 (HWASA)
7위 | Drowning - WOODZWOODZ
8위 | 뛰어(JUMP) - BLACKPINKBLACKPINK
9위 | 어제보다 슬픈 오늘 - 우디 (Woody)우디 (Woody)
10위 | Soda Pop - KPop Demon Hunters C
11위 | IRIS OUT - Kenshi YonezuKenshi 
12위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)박다혜
13위 | FAMOUS - ALLDAY PROJECTALLDAY
14위 | XOXZ - IVE (아이브)IVE (아이브)
15위 | Hollywood Action - BOYNEXTDOORBOYNEXTDO
16위 | Rich Man - aespaaespa
17위 | 모르시나요(PROD.로코베리) - 조째즈조째즈
18위 | 한번 더 이별 - 이창섭이창섭
19위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴로이킴
20위 | body - 다영 (DAYOUNG)다영 (DAYO
21위 | 너에게 닿기를 - 10CM10CM
22위 | 운명 (2025) - 먼데이 키즈, 이이경먼데이 키즈, 이
23위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGONG-DRAGON
24위 | toxic till the end - 로제 (ROSÉ)로제 (ROSÉ)
25위 | Whiplash - aespaaespa
26위 | FOCUS - Hearts2Hearts (하츠투하츠
27위 | 시작의 아이 - 마크툽 (MAKTUB)마크툽 (MAK
28위 | 청춘만화 - 이무진이무진
29위 | 천상연 - 이창섭이창섭
30위

### 5) 네이버 지식인으로 검색(open API 사용X)
- 특정 keyword를 특정페이지 수만큼

In [10]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '쳇지피티'
url = f'https://kin.naver.com/search/list.naver'
params = {'query' : keyword}
response = get(url, params=params)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')
# soup

200


In [26]:
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
keyword = quote('쳇지피티')
pages = 3
items_list = [] # 크롤링한 데이터를 담을 list
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    response = urlopen(url)
    # print(response.status)
    soup = BeautifulSoup(response, 'html.parser')
    items = soup.select('dt > a')
    for item in items:
        title = item.text
        link = item.attrs.get('href')
        items_list.append({
            'title':title,
            'link':link
        })
df = pd.DataFrame(items_list)
df

,title,link
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...
1,쳇지피티 관해서,https://kin.naver.com/qna/detail.naver?d1id=1&...
2,부족한지 봐주세요(쳇지피티 답변 금지),https://kin.naver.com/qna/detail.naver?d1id=3&...
3,쳇지피티 유료로 전환시,https://kin.naver.com/qna/detail.naver?d1id=1&...
4,쳇지피티 3일 리셋,https://kin.naver.com/qna/detail.naver?d1id=8&...
5,쳇지피티 구글계정 목록,https://kin.naver.com/qna/detail.naver?d1id=1&...
6,쳇지피티 문법,https://kin.naver.com/qna/detail.naver?d1id=11...
7,고등학교 수행평가에 쳇지피티 쓰면...,https://kin.naver.com/qna/detail.naver?d1id=11...
8,쳇지피티 이혼수 있데요,https://kin.naver.com/qna/detail.naver?d1id=3&...
9,제가 직접 쓴 자소서 쳇지피티,https://kin.naver.com/qna/detail.naver?d1id=4&...


## 2.2 openAPI사용 : json 웹데이터 수집
### 1) 네이버 지식인으로 검색 (open API 사용 O)
- 네이버 개발자센터에서 애플리케이션 등록(이름/검색/WEB설정 http://localhost)
- .env파일에 CLIENT_ID/CLIENT_SECRET 환경변수 저장
- 환경변수를 읽기 위해서 `pip install dotenv`
- 특정 keyword를 지식검색(데이터수 30개)

In [28]:
# 환경변수 읽어오기
from dotenv import load_dotenv
import os
load_dotenv()
print(os.getenv('CLIENT_ID'))
print(os.getenv('CLIENT_SECRET'))

OXJ2flFeL54mJJlKjKaA
Q8xLh5gudn


In [30]:
# 방법2
import os
import urllib.request
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("쳇지피티")
url = "https://openapi.naver.com/v1/search/kin?query=" + encText # JSON 결과
# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
request = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(request)
rescode = response.status
if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8')))
    print(response_body.decode('utf-8')[:50])
else:
    print("Error Code:" + rescode)

<class 'str'>
{
	"lastBuildDate":"Tue, 11 Nov 2025 11:57:43 +090


In [41]:
# 방법1
import requests
import pandas as pd
import json # response텍스트를 json스타일의 딕셔너리로
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query = '쳇지피티'
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
# url = f'https://openapi.naver.com/v1/search/kin?query={query}&display=30'
# response = requests.get(url, headers=headers)
url = 'https://openapi.naver.com/v1/search/kin'
params = {'query':query, 'display':30}
response = requests.get(url, params=params, headers=headers)
# print(response.text[:500])
# items = json.loads(response.text)['items']
items = response.json()['items']
items_list = []
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link  = item.get('link')
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link, description])
pd.DataFrame(items_list, columns=['title', 'link', 'description']).head()

,title,link,description
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...,쳇지피티 사주 정확성 몇프로 정도 될까요? 큰틀은 얼추 맞는거 같기도하고 모르겠네요...
1,쳇지피티 관해서,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티한테 질문 제대로 하려면 어떻게 해야 하냐요?... 쳇지피티 한테 하는질문도...
2,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지),https://kin.naver.com/qna/detail.naver?d1id=3&...,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지) 남매입니다 남자 1986년 9...
3,쳇지피티 유료로 전환시,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 무료버전으로 카톡대화창 전송해서 상대의심리에 관한 분석요청을 많이했는데 첨...
4,쳇지피티 3일 리셋,https://kin.naver.com/qna/detail.naver?d1id=8&...,... 쳇지피티 어떻게 하는건가욤 유료결제인가요 유료결제는 얼마인가요 그거 하게되면...


### quiz) 네이버 openAPI를 이용해서 청바지 이미지 100건의 데이터를 ch14_청바지.csv로 출력
```
제목, 링크, 썸네일, sizeheight, sizewidth
```

In [56]:
import requests
import pandas as pd
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query = '청바지'
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
url = 'https://openapi.naver.com/v1/search/image'
params = {'query':query, 'display':100}
response = requests.get(url, params=params, headers=headers)
items = response.json()['items']
items_list = []
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link  = item.get('link')
    img = item.get('thumbnail')
    sizeheight=300
    sizewidth=300
    items_list.append([title, link, img])
imglink = pd.DataFrame(items_list, columns=['title', 'link', 'img']).head()
imglink.to_csv(f'data/ch14_{query}.csv', index=False)

In [57]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법2)'
    from urllib.request import urlopen, Request
    from urllib.parse import quote
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    encText = quote(query)
    url = f'https://openapi.naver.com/v1/search/image?query={encText}&display=100'
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    request = Request(url, headers=headers)
    response = urlopen(request)
    # print(response.read().decode('utf-8'))
    items = json.loads(response.read().decode('utf-8'))['items']
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

,제목,링크,썸네일,sizeheight,sizewidth
4,JEEP spirit 남성 청바지 스판 남자 캐쥬얼 팬츠 지프 : 우수한 편의점,https://shop-phinf.pstatic.net/20251014_131/17...,https://search.pstatic.net/common/?type=b150&s...,800,800


In [ ]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

In [69]:
def get_image_list_save_image(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = [] # 정보가 담길 리스트
    for idx, item in enumerate(items):
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        # 이미지 파일 저장
        save_image('메인', idx, link, query)
        save_image('썸네일', idx, thumbnail, query)
        if (idx%20 == 0) & (idx!=0):
            print(f'= = = {idx}% 진행중 = = =')
    print('= = = 완료 = = =')
    return pd.DataFrame(items_list)
df = get_image_list_save_image("청바지")
df.to_csv('image/ch14_청바지.csv')
df.sample()

= = = 20% 진행중 = = =
= = = 40% 진행중 = = =
= = = 60% 진행중 = = =
= = = 80% 진행중 = = =
= = = 완료 = = =


,제목,링크,썸네일,sizeheight,sizewidth
11,코아픽샵 남자 키작남 일자핏 청바지 도톰 스트레이트 스판 9부 바지 꾹1 : 코아픽샵,https://shop-phinf.pstatic.net/20251004_170/17...,https://search.pstatic.net/common/?type=b150&s...,500,375


In [59]:
print(df.loc[99, '링크'])
print(df.loc[99, '링크'])

http://shop1.phinf.naver.net/20251012_221/1760195026829GJCj4_JPEG/94327887959887005_1382895326.jpeg
http://shop1.phinf.naver.net/20251012_221/1760195026829GJCj4_JPEG/94327887959887005_1382895326.jpeg


In [67]:
def save_image(attr, idx, link, query):
    'link의 이미지를 image/attr_idx_query.확장자로 local에 저장'
    import requests
    file_extension = link.split('.')[-1] # 확장자
    quote_index = file_extension.find('?') # 확장자뒤에 ?가 있는 위치
    if quote_index != -1:
        file_extension = file_extension[:quote_index]
    img = requests.get(link).content # 바이너리
    with open(f'image/{attr}_{idx+1}_{query}.{file_extension}', 'wb') as f:
        f.write(img)
save_image('메인', 0, df.loc[0, '링크'], '청바지')

## 2.3 XML 웹데이터 수집
- RSS / open API를 통한 XML 웹데이터 수집
### 1) 전국 날씨 RSS를 BeautifalSoup을 이용한 크롤링
-구글에 "기상청 RSS" 검색한 첫번째 사이트 이동

In [70]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
items_list = []
url = 'https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml'
# 방법1
# target = requests.get(url)
# soup = BeautifulSoup(target.text, "xml") # pip install lxml 필요없음
# 방법2
target = urlopen(url)
soup = BeautifulSoup(target, "xml")
local_tas = soup.select('local_ta')
for local_ta in local_tas:
    local_name = local_ta.select_one('local_ta_name').text
    week1_normalYear = local_ta.select_one('week1_local_ta_normalYear').text
    week1_similarRange = local_ta.select_one('week1_local_ta_similarRange').text
    week1_minVal = local_ta.select_one('week1_local_ta_minVal').text
    week1_similarVal = local_ta.select_one('week1_local_ta_similarVal').text
    week1_maxVal = local_ta.select_one('week1_local_ta_maxVal').text
    items_list.append({
        '지역':local_name,
        '1주평년기온':week1_normalYear,
        '1주기온범위':week1_similarRange,
        '1주낮을확률':week1_minVal,
        '1주비슷할확률':week1_similarVal,
        '1주높을확률':week1_maxVal
    })
df = pd.DataFrame(items_list)
df['1주평년기온'] = df['1주평년기온'].astype(np.float64)
df['1주낮을확률'] = df['1주낮을확률'].astype(np.int16)
df['1주비슷할확률'] = df['1주비슷할확률'].astype(np.int16)
df['1주높을확률'] = df['1주높을확률'].astype(np.int16)
df.head()

,지역,1주평년기온,1주기온범위,1주낮을확률,1주비슷할확률,1주높을확률
0,"전국(제주도,북한제외)",5.9,5.0~6.8,20,40,40
1,서울ㆍ인천ㆍ경기도,4.8,3.7~5.9,20,40,40
2,강원도 영서,2.6,1.5~3.7,20,40,40
3,강원도 영동,5.8,4.9~6.7,20,40,40
4,대전ㆍ세종ㆍ충청남도,5.4,4.5~6.3,20,40,40


### 2) XML응답하는 open API 활용
- data.go.kr에서
    - 서울특별시_노선정보조회 서비스(버스ID, 정류장목록과 정류장ID)
    - 서울특별시_버스위치정보조회 서비스(실시간 버스 위치 목록)

In [18]:
# STEP 1 버스번호의 버스 id받아오기
# 서울특별시_노선정보조회 서비스 - 3번 기능(getBusRouteList) 이용
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('KEY'))
# print(os.getenv('KEY1'))

True

In [20]:
from bs4 import BeautifulSoup
# urlretrieve(url, 저장경로) : url의 파일을 저장경로에 저장
from urllib.request import urlretrieve
from urllib.parse import quote
# busNum = quote('구로09')
busNum = '270'
key = os.getenv('KEY')
url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
print(url)
savefilename1 = 'data/ch14_busInfo.xml'
urlretrieve(url, savefilename1) # xml파일(url)을 savefilename으로 저장
with open(savefilename1, encoding='utf-8') as f:
    xml = f.read();
soup = BeautifulSoup(xml, 'xml')

http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey=693f6b4e2b9cfa42ebd3871d65a4fde0319d263007fc2312aeb2e75cecef29aa&strSrch=270


In [21]:
import requests
from urllib.parse import quote
# busNum = quote('구로09')
busNum = '270'
key = os.getenv('KEY')
url1 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
response = requests.get(url1)
soup = BeautifulSoup(response.text, 'xml')

for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm == busNum:
        busRouteId = item.select_one('busRouteId').text
        break
print('busRouteId =', busRouteId)

busRouteId = 100100046


In [ ]:
# step2. 버스id로 버스정류장목록 받아오기(정류장명, 정류장id, 경도, 위도)

In [22]:
pd.options.display.max_rows = 60

In [23]:
import pandas as pd
url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
response = requests.get(url2)
soup = BeautifulSoup(response.content, 'xml')
itemLists = soup.find_all('itemList')
print(f'{busNum}번 정류장 갯수 : {len(itemLists)}')
bus_station = []
for itemList in itemLists:
    stationNm = itemList.find('stationNm').text # 정류소 이름
    station   = itemList.find('station').text   # 정류소 id
    gpsX      = itemList.find('gpsX').text      # 경도
    gpsY      = itemList.find('gpsY').text      # 위도
    bus_station.append([stationNm, station, gpsX, gpsY])
df_station = pd.DataFrame(bus_station, columns=['정류소명', 'id', '경도', '위도'])
df_station.head()

270번 정류장 갯수 : 98


,정류소명,id,경도,위도
0,월드컵파크7단지,113000180,126.8784199476,37.5811497135
1,구룡근린공원,113000196,126.8826124496,37.5834310105
2,DMC첨단산업센터,113000021,126.885192,37.585347
3,수색교,111000003,126.888672,37.585759
4,수색역앞,111000005,126.8932380184,37.5832230348


In [14]:
import pandas as pd
url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
response = requests.get(url2)
soup = BeautifulSoup(response.content, 'xml')
itemLists = soup.find_all('itemList')
print(f'{busNum}번 정류장 갯수 : {len(itemLists)}')
bus_station = []
for itemList in itemLists:
    stationNm = itemList.find('stationNm').text # 정류소 이름
    station   = itemList.find('station').text   # 정류소 id
    gpsX      = itemList.find('gpsX').text      # 경도
    gpsY      = itemList.find('gpsY').text      # 위도
    bus_station.append([stationNm, station, gpsX, gpsY])
df_station = pd.DataFrame(bus_station, columns=['정류소명', 'id', '경도', '위도'])
df_station.head()

270번 정류장 갯수 : 98


,정류소명,id,경도,위도
0,월드컵파크7단지,113000180,126.8784199476,37.5811497135
1,구룡근린공원,113000196,126.8826124496,37.5834310105
2,DMC첨단산업센터,113000021,126.885192,37.585347
3,수색교,111000003,126.888672,37.585759
4,수색역앞,111000005,126.8932380184,37.5832230348


In [ ]:
# step3. 버스id로 실시간위치정보를 받아오기(차량번호, 혼잡도, 최종정류장id, 다음정류장id, 도착소요시간)

In [34]:
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey={key}&busRouteId={busRouteId}'
print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text, 'xml')
itemLists = soup.select('itemList')
print(f'운행중인 버스는 {len(itemLists)}대입니다')
bus_position = []
for itemList in itemLists:
    plainNo = itemList.select_one('plainNo').text # 차량번호
    congetion = itemList.select_one('congetion').text # 혼잡도(congetion)
    # 0 : 없음, 3 : 여유, 4 : 보통, 5 : 혼잡, 6 : 매우혼잡
    congetion = '없음' if congetion=='0' \
            else '여유' if congetion=='3' \
            else '보통' if congetion=='4' \
            else '혼잡' if congetion=='5' else '매우혼잡'
    gpsX = itemList.select_one('gpsX').text # 경도
    gpsY = itemList.select_one('gpsY').text # 위도
    lastStnId = itemList.select_one('lastStnId').text # 최종정류소id
    nextStId = itemList.select_one('nextStId').text # 다음정류소id
    nextStTm = itemList.select_one('nextStTm').text # 다음정류소까지 소요시간
    
    bus_position.append({
        '차량번호':plainNo,
        '혼잡도':congetion,
        '경도':gpsX,
        '위도':gpsY,
        '이전정류소id':lastStnId,
        '다음정류소id':nextStId,
        '도착소요시간':nextStTm
    })
df_position = pd.DataFrame(bus_position)
df_position.head(2)

http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey=693f6b4e2b9cfa42ebd3871d65a4fde0319d263007fc2312aeb2e75cecef29aa&busRouteId=100100046
운행중인 버스는 26대입니다


,차량번호,혼잡도,경도,위도,이전정류소id,다음정류소id,도착소요시간
0,서울70사8023,여유,126.87842,37.58115,113000180,111000005,352
1,서울74사9503,여유,126.882047,37.583293,113000180,111000005,357


In [27]:
df_station.loc[df_station['id']=='113000180','정류소명'].iloc[0]

'월드컵파크7단지'

In [28]:
df_station.loc[df_station['id']=='111000005','정류소명'].iloc[0]

'수색역앞'

In [35]:
def station_name(row):
    row = row.copy()
    'row의 이전정류소명과 다음정류소명을 추가하고, 도착소요시간을 분으로 바꿔 return'
    row['이전정류소명'] = df_station.loc[df_station['id']==row['이전정류소id'],'정류소명'].iloc[0]
    row['다음정류소명'] = df_station.loc[df_station['id']==row['다음정류소id'],'정류소명'].iloc[0]
    minite = int(row['도착소요시간'])//60
    sec    = int(row['도착소요시간']) % 60
    row['도착소요시간'] = f'{minite}분{sec}초'
    return row

In [37]:
df_position = df_position.apply(station_name, axis=1)
df_drop(['이전정류소id','다음정류소id'], axis=1).head()

ValueError: invalid literal for int() with base 10: '5분52초'

# 3장 연습문제(quiz1)
```
yes24 의 베스트셀러 정보를 제공하는 사이트에서 베스트셀러 정보를
수집해서 파일에 저장하세요
베스트셀러 정보 수집 주소 : http://www.yes24.com/24/category/bestseller
```

In [115]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [123]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
url = 'http://www.yes24.com/24/category/bestseller'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
book_find_list = []
book_rank_els = soup.select('div.img_upper > em.ico.rank')
book_rank = [int(book_rank_el.text) for book_rank_el in book_rank_els]
book_name_els = soup.select('div.info_row > a.gd_name')
book_name = [book_name_el.text for book_name_el in book_name_els]
authors_els = soup.select('span.authPub.info_auth')
authors = [authors_el.text.strip() for authors_el in authors_els]
publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
publishers = [publishers_el.text for publishers_el in publishers_els]
book_price_els = soup.select('div.info_price > strong.txt_num')
book_price = [book_price_el.text for book_price_el in book_price_els]

with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
    for rank, title, publisher, price in zip(book_rank,book_name,publishers,book_price):
        f.write(f'{rank}, {title}, {publisher}, {price}\n')
        book_find_list.append({rank, title, publisher, price})
df = pd.DataFrame(book_find_list, columns=['순위','책이름','출판사','가격'])
df.to_csv(f'data/ch14_yes24.csv', index=False)
df

,순위,책이름,출판사,가격
0,1,"18,000원",미래의창,트렌드 코리아 2026
1,"17,100원",북플레저,2,처음 만나는 양자의 세계
2,사피엔테스,대형주 추세추종 투자법칙,3,"19,800원"
3,위버멘쉬,떠오름,4,"16,020원"
4,박곰희 연금 부자 수업,5,"18,900원",인플루엔셜
5,주식 투자의 뿌리,"19,800원",6,애덤스미스
6,"16,200원",절창 切創,문학동네,7
7,8,다크 심리학,"19,710원",어센딩
8,9,포레스트북스,"21,420원",단 한 줄만 내 마음에 새긴다고 해도
9,페이지2북스,10,"16,020원",어른의 행복은 조용하다


In [116]:
# 방법1
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # 순위
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    # 책제목
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    # 저자
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.strip() for authors_el in authors_els]
    for i, author in enumerate(authors):
        match = re.search(r"(.*)외 \d+명", author)
        if(match):
            authors[i] = match.group()
            print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    # 출판사
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    # 가격
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head()

,순위,책이름,저자,출판사,가격
0,25,부자 아빠 투자 불변의 법칙,타짱 저/박선영 역,큰숲,"18,000원"
1,26,진돗개 두 마리와 네덜란드에서 살고 있습니다,박혜령 저,위즈덤하우스,"16,200원"
2,27,류수영의 평생 레시피,류수영 저,세미콜론,"22,500원"
3,28,달러 자산 1억으로 평생 월급 완성하라 (2026 투자 전략 특별판),채부심(채상욱) 저,몽스북,"18,900원"
4,29,그 비스크 돌은 사랑을 한다 15,후쿠다 신이치 글그림/박연지 역,소미미디어,"5,400원"


In [124]:
# 방법2
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = urlopen(url)
    soup = BeautifulSoup(response, 'html.parser')
    # 순위
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    # 책제목
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    # 저자
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.split('정보 더 보기/감추기')[0].strip() 
                                   for authors_el in authors_els]
#     for i, author in enumerate(authors):
#         match = re.search(r"(.*)외 \d+명", author)
#         if(match):
#             authors[i] = match.group()
#             print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    # 출판사
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    # 가격
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)

,순위,책이름,저자,출판사,가격
0,1,트렌드 코리아 2026,"김난도, 전미영, 최지혜, 권정윤, 한다혜 저 외 7명",미래의창,"18,000원"
